<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings # Импорт модуля warnings необходим, чтобы игнорировалось предупреждение о необходимости использовать SQLAlchemy
warnings.filterwarnings('ignore')

In [2]:
# Параметры подключения:
DBNAME = 'xxx'
USER = 'xxx'
PASSWORD = 'xxx'
HOST = 'xxx'
PORT = xxx

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
# Считаем столбцы id в таблице vacancies
query_3_1 = f'''select
                    count(id) /*Считаем количество записей в столбце id*/
                from public.vacancies /*Из таблицы vacancies*/
            '''

In [5]:
# результат запроса
vacancies_number = pd.read_sql_query(query_3_1, connection)
vacancies_number

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
# Считаем столбцы id в таблице employers
query_3_2 = f'''select
                    count(id) /*Считаем количество записей в столбце id*/
                from public.employers /*Из таблицы employers*/
            '''

In [7]:
# результат запроса
employers_number = pd.read_sql_query(query_3_2, connection)
employers_number

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
# Считаем столбцы id в таблице areas
query_3_3 = f'''select
                    count(id) /*Считаем количество записей в столбце id*/
                from public.areas /*Из таблицы areas*/
            '''

In [9]:
# результат запроса
areas_number = pd.read_sql_query(query_3_3, connection)
areas_number

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
# Считаем столбцы id в таблице industries
query_3_4 = f'''select
                    count(id) /*Считаем количество записей в столбце id*/
                from public.industries /*Из таблицы industries*/
            '''

In [11]:
# результат запроса
industries_number = pd.read_sql_query(query_3_4, connection)
industries_number

,count
0,294


***

Предварительный анализ данных, содержащихся в базе, показывает, что в ней представлены 49197 вакансий от 23501 работодателя из 1362 регионов, охватывающих 294 сферы деятельности. То есть в среднем чуть более двух вакансий от работодателя. На каждый регион приходится в среднем более 17 работодателей и более 36 вакансий. Каждая сфера представлена в среднам чуть менее 80 работодателями и предоставляет в среднем чуть более 167 вакансий.

Естественно, что преиведённые выше числа весьма грубы, потому что каждый регион обладает различными ресурсами в плане предложения рабочих мест, равно как и не все сферы деятельности одинаково востребованы на рынке труда.

Поэтому необходим более глубокий анализ данных.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''SELECT 
                    COUNT(v.id) cnt, /*Считаем количество записей в столбце id таблицы v, присваиваем солбцу название cnt*/
                    a.name area /*Выбираем столбец name таблицы a, присваиваем солбцу название area*/
                FROM public.vacancies v /*Таблица vacancies с алиасом v*/
                JOIN public.areas a ON v.area_id = a.id /*Присоединяемая таблица areas с алиасом a; присоединение по столбцам area_id таблицы v и id таблицы а*/
                GROUP BY a.name /*Группировка по столбцу name таблицы а*/
                ORDER BY COUNT(v.id) DESC /*Сортировка по подсчёту id таблицы v; сортировка по убыванию*/
                LIMIT 5 /*Выводим первые пять строк*/
            '''

In [13]:
# результат запроса
vacancies_per_region = pd.read_sql_query(query_4_1, connection)
vacancies_per_region

,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''SELECT
                    COUNT(id) /*Считаем количество записей в столбце id...*/
                FROM public.vacancies /*... таблицы vacancies*/
                WHERE (salary_from is not null) or (salary_to is not null) /*где значения столбца salary_from или salary_to ненулевые*/
            '''

In [15]:
# результат запроса
not_empty_salary = pd.read_sql_query(query_4_2, connection)
not_empty_salary

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''SELECT
                    ROUND(AVG(salary_from)) low_level, /*Считаем среднее по столбцу salary_from, округляя его до целого; присваиваем название столбцу low_level*/
                    ROUND(AVG(salary_to)) upper_level /*Считаем среднее по столбцу salary_to, округляя его до целого; присваиваем название столбцу upper_level*/
                FROM public.vacancies /*Из таблицы vacancies*/
                WHERE (salary_from is not null) or (salary_to is not null) /*где значения столбца salary_from или salary_to ненулевые*/
            '''

In [17]:
# результат запроса
average_salary = pd.read_sql_query(query_4_3, connection)
average_salary

,low_level,upper_level
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''SELECT
                    COUNT(id), /*Считаем количество записей в столбце id*/
                    schedule,
                    employment /*Выбираем столбцы schedule и employment*/
                FROM public.vacancies /*Из таблицы vacancies*/
                GROUP BY schedule, employment /*Группируем по столбцам schedule и employment*/
                ORDER BY COUNT(id) DESC /*Сортируем по убыванию по столбцу с подсчётом id*/
            '''

In [19]:
# результат запроса
vac_scheldue_employment = pd.read_sql_query(query_4_4, connection)
vac_scheldue_employment

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''SELECT
                    experience /*Выбираем столбец experience*/
                FROM public.vacancies /*Из таблицы vacancies*/
                GROUP BY experience /*Группируем по столбцу experience*/
                ORDER BY COUNT(id) /*Сортируем по подсчёту записей в столбце id*/
            '''

In [21]:
# результат запроса
experience_for_vacancies = pd.read_sql_query(query_4_5, connection)
experience_for_vacancies

,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

На основании детального анализа вакансий можно сделать следующие выводы:

1. Как и следовало ожидать, лидерами по количеству вакансий являются столицы (Москва, Минск, отчасти сюда же можно отнасти и С.-Петербург) и города-миллионники (Новосибирск и Алматы, тоже бывшая столица).

2. Из 49197 вакансий хотя бы одно поле с зарплатой заполнено всего лишь у 24073, т.е. меньше, чем у половины. Причин тому может несколько:
 
   а) работодатель сам не определился с зуровнем заработной платы, которую готов предложить кандидату;

   б) уровень зароботной платы зависит от уровня кандидата, а сам этот уровень будет выявляться на техническом собеседовании;

   в) причина может быть чисто психологической: работодатель может скрывать зарплату от соискателей не от самих соискателей, а от уже работающих в данной компании, чтобы не унинзить их в случае если они работают за меньшие деньги.

3. Средняя минимальная зарплата - 71 тыс. руб. Средняя максимальная - 110,5 тыс. руб. Эти цифры вполне соответствуют рынку и коррелируются с показателями, которые будут озвучены в п.5: для кандидатов с опытом работы от 1 года до 3 лет такое предложение вполне соответствует современным требованиям рынка, тем более что окончательная заработная плата, естественно, устанавливается, как правило, после соответствующих переговоров между работодателем и кандидатом.

4. В общей сложности 46069 вакансий (93,6% от общей численности) подразумевают полную занятость. Из них 35367 вакансий (71,9% от общей численности) предполагают ещё и полный рабочий день. Данные цифры говорят о серьёзности намерений работодателей, их желании нанять работников на постоянную работу, что предполагает высокие требования к кандидатам.

5. А вот требуемый опыт работы говорит, скорее, о дефиците рабочих кадров, поскольку наиболее часто встречающееся требование - от 1 года до 3 лет - указывает на то, что кампании готовы нанимать работников с минимальным опытом работы. С другой стороны, готовность нанять работника с минимальным опытом может говорить и о желании компании вкладываться в его переобучение под нужды самой кампании, поскольку прежние навыки работника могут быть нерелевантны для его нового работодателя.



# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''(SELECT 
                    e.name, /*Выбираем столбец name таблицы е*/
                    COUNT(v.id) /*Считаем количество записей в столбце id таблицы v*/
                FROM public.vacancies v /*Из таблицы vacancies с алиасом v*/
                JOIN public.employers e ON v.employer_id = e.id /*присоединяем таблицу employers с алиасом е; соединение по столбцам employer_id таблицы v и id таблицы е*/
                GROUP BY e.name /*Группируем по столбцу name таблицы е*/
                ORDER BY COUNT(v.id) DESC /*Сортируем по убыванию по столбцу с подсчётом записей в столбце id таблицы v*/
                LIMIT 1) /*Выводим первую строчку*/
                
                UNION /*Оператор объединения запросов*/
                
                (SELECT /*Запрос, аналогичный первому*/
                    e.name,
                    COUNT(v.id)
                FROM public.vacancies v
                JOIN public.employers e ON v.employer_id = e.id
                GROUP BY e.name
                ORDER BY COUNT(v.id) DESC
                OFFSET 4 LIMIT 1) /*Пропускаем первые 4 записи и выводим первую из оставшихся*/

                ORDER BY 2 DESC /*Всю итоговую таблицу сортируем по убыванию по второму столбцу*/
            '''

In [23]:
# результат запроса
first_and_fifth_employers = pd.read_sql_query(query_5_1, connection)
first_and_fifth_employers

,name,count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''(SELECT 
                    a.name, /*Выбираем столбец name таблицы а*/
                    COUNT(DISTINCT e.id) employers, /*Считаем количество уникальных записей в столбце id таблицы е; присваиваем столбцу название employers*/
                    v.id vacancies /*Выбираем столбец id таблицы v; присваиваем столбцу название vacancies*/
                FROM public.areas a /*Из таблицы areas с алиасом a*/
                LEFT JOIN public.vacancies v ON a.id = v.area_id /*присоединяем методом LEFT JOIN таблицу vacancies с алиасом v; соединение по столбцам id таблицы а и area_id таблицы v*/
                LEFT JOIN public.employers e ON a.id = e.area /*присоединяем методом LEFT JOIN таблицу employers с алиасом е; соединение по столбцам id таблицы а и area таблицы е*/
                GROUP BY a.name, v.id /*Группируем по столбцам name таблицы а и id таблицы v*/
                HAVING v.id IS NULL /*При условии, что значения в столбце v.id ненулевые*/
                ORDER BY 2 DESC /*Сортируем по убыванию по столбцу 2*/
                LIMIT 1) /*Выводим первую строчку*/

                UNION ALL /*Оператор объединения запросов*/

                (SELECT 
                    a.name, /*Выбираем столбец name таблицы а*/
                    COUNT(DISTINCT v.employer_id) employers, /*Считаем количество уникальных записей в столбце employer_id таблицы v; заносим значения в столбец employers*/
                    COUNT(v.id) vacancies /*Считаем количество записей в столбце id таблицы v; заносим значения в столбец vacancies*/
                FROM public.vacancies v /*Из таблицы vacancies с алиасом v*/
                JOIN public.areas a ON v.area_id = a.id /*присоединяем таблицу areas с алиасом а; соединение по столбцам area_id таблицы v и id таблицы а*/
                GROUP BY a.name) /*Группируем по столбцу name таблицы а*/
            '''

In [25]:
# результат запроса
regions_on_employers_and_vacancies = pd.read_sql_query(query_5_2, connection)
regions_on_employers_and_vacancies

,name,employers,vacancies
0,Россия,410,NaN
1,Абакан,25,37.0
2,Абан,1,1.0
3,Абдулино,1,1.0
4,Абинск,3,6.0
...,...,...,...
765,Ялуторовск,3,3.0
766,Ярославль,198,391.0
767,Ярцево,2,2.0
768,Ясногорск,2,2.0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# текст запроса
query_5_3 = f'''SELECT 
                    e.name, /*Выбираем столбец name таблицы е*/
                    COUNT(DISTINCT v.area_id) /*Считаем количество уникальных записей в столбце area_id таблицы v*/
                FROM public.employers e /*Из таблицы employers с алиасом e*/
                JOIN public.vacancies v ON e.id = v.employer_id /*присоединяем таблицу vacancies с алиасом v; соединение по столбцам id таблицы е и employer_id таблицы v*/
                GROUP BY e.name /*Группируем по столбцу name таблицы е*/
                ORDER BY 2 DESC /*Сортируем по убыванию по столбцу 2*/
            '''

In [27]:
# результат запроса
employers_in_regions = pd.read_sql_query(query_5_3, connection)
employers_in_regions

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
# текст запроса
query_5_4 = f'''SELECT 
                    COUNT(DISTINCT e.id) /*Считаем количество уникальных записей в столбце id таблицы е*/
                FROM public.employers e /*Из таблицы employers с алиасом e*/
                LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id /*присоединяем методом LEFT JOIN таблицу employers_industries с алиасом ei; соединение по столбцам id таблицы е и employer_id таблицы ei*/
                WHERE ei.industry_id IS NULL /*Где значения в столбце ei.industry_id ненулевые*/
            '''

In [29]:
# результат запроса
employers_without_industry = pd.read_sql_query(query_5_4, connection)
employers_without_industry

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# текст запроса
query_5_5 = f'''SELECT 
                    e.name /*Выбираем столбец name таблицы е*/
                FROM public.employers e /*Из таблицы employers с алиасом e*/
                JOIN public.employers_industries ei ON e.id = ei.employer_id /*присоединяем таблицу employers_industries с алиасом ei; соединение по столбцам id таблицы е и employer_id таблицы ei*/
                GROUP BY e.name /*Группируем по столбцу name таблицы е*/
                HAVING COUNT(DISTINCT ei.industry_id) = 4 /*При условии, что количество уникальных записей в столбце ei.industry_id равно 4*/
                ORDER BY e.name /*Сортируем по столбцу e.name*/
                OFFSET 2 LIMIT 1 /*Пропускаем первые 2 записи и выводим первую из оставшихся*/
            '''

In [31]:
# результат запроса
employers_with_four_industries = pd.read_sql_query(query_5_5, connection)
employers_with_four_industries

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
# текст запроса
query_5_6 = f'''SELECT 
                    COUNT(DISTINCT ei.employer_id) /*Считаем количество уникальных записей в столбце employer_id таблицы ei*/
                FROM public.employers_industries ei /*Из таблицы employers_industries с алиасом ei*/
                JOIN public.industries i ON ei.industry_id = i.id /*присоединяем таблицу eindustries с алиасом i; соединение по столбцам industry_id таблицы ei и id таблицы i*/
                WHERE i.name = 'Разработка программного обеспечения' /*Где значения в столбце i.name 'Разработка программного обеспечения'*/
            '''

In [33]:
# результат запроса
software_development = pd.read_sql_query(query_5_6, connection)
software_development

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
# код для получения списка городов-милионников
import urllib.parse as req # Импортируем библиотеку urllib.parse для парсинга таблицы с городами-миллионниками из "Википедии"

# Избавляемся от кириллицы в пути URL:
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров') 
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)

# Получаем таблицу в формте DF:
table = pd.read_html(url)[1]

# Из таблицы получаем кортеж с городами-миллионниками:
cities_millionairs = tuple(table['Город'])
cities_millionairs

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [35]:
# текст запроса
query_5_7 = f'''WITH Temp AS( /*Формируем временный запрос под названием Temp*/
                SELECT 
                    a.name city, /*Выбираем столбец name таблицы а; присваиваем столбцу название city*/
                    COUNT(v.id) cnt /*Считаем количество записей в столбце id таблицы v; присваиваем столбцу название cnt*/
                FROM public.areas a /*Из таблицы areas с алиасом а*/
                JOIN public.vacancies v ON a.id = v.area_id /*присоединяем таблицу vacancies с алиасом v; соединение по столбцам id таблицы а и area_id таблицы v*/
                JOIN public.employers e ON v.employer_id = e.id
                WHERE e.name = 'Яндекс' /*Где значения в столбце е.name 'Яндекс'*/
                GROUP BY a.name /*Группируем по столбцу name таблицы а*/
                HAVING a.name in {cities_millionairs} /*При условии, что запись в столбце a.name есть в переменной cities_millionairs*/
                ORDER BY cnt /*Сортируем по столбцу cnt*/
                )
                
                SELECT * /*Выбираем все столбцы*/
                FROM Temp /*из таблицы Temp*/
                
                UNION ALL /*Оператор объединения запросов*/
                
                SELECT 'total', sum(cnt) /*Вводим в первый столбец запись 'total', а во второй - сумму по столбцу cnt*/
                FROM Temp /*из таблицы Temp*/
            '''

In [36]:
# результат запроса
yandex_vacancies = pd.read_sql_query(query_5_7, connection)
yandex_vacancies

,city,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

Анализ работодателей показал следующее.

1. Лидерами по количеству выставленных вакансий являются компании, развивающие IT-направления. Это неудивительно, поскольку отрасль IT является одним из локомотивов современной экономики.

2. Наибольшее количество работодателей находится в столичных городах и городах-миллионниках. Это подчёркивает тенденцию к урбанизации, характерную для современного этапа развития экономики.

3. Крупные компании (такие, как "Яндекс", "Ростелеком", "Совкомбанк" и т.п.) публикуют свои вакансии в наибольшем количестве регионов. Это вполне естественно, поскольку филиалы таких компаний находятся по всей стране.

4. В общей сложности 8419 компаний не указали сферу деятельности, в которую они хотят нанять сотрудников. Причин тому, как представляется, может быть две: либо сфера деятельности компании понятна из её названия, либо же от кандидата требуется слишком широкий набор навыков, который трудно уложить в два-три слова. Вариант, при котором работодатель может скрывать свою сферу деятельности, обсуждать не хочется.

5. Очень много работодателей указали 4 и более сфер деятельности. Это говорит о востребованности на рынке труда работников с самыми разнообразными навыками.

6. 3553 работодателя ищут соискателей в сферу разработки программного обеспечения. Это лишний раз подтверждает высокий спрос на представителей IT-специальнотсей, о котором говорилось в п.1.

7. Отечественный лидер IT-разрабток, компания "Яндекс", выставила 485 вакансий в городах-миллионниках. Данный факт свидетельствует о том, что у компании всё хорошо, она расширяет штат сотрудников в связи с ростом объёма работ.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# текст запроса
query_6_1 = f'''SELECT 
                    COUNT(name) /*Считаем количество записей в столбце name*/
                FROM public.vacancies /*Из таблицы vacancies*/
                WHERE lower(name) LIKE '%data%' OR lower(name) LIKE '%данн%' /*Где значения в столбце, приведённые к нижнему регистру, содержат комбинацию символов 'data' и 'данн'*/
            '''

In [38]:
# результат запроса
data_vacancies = pd.read_sql_query(query_6_1, connection)
data_vacancies

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
# текст запроса
query_6_2 = f'''SELECT 
                    COUNT(name) /*Считаем количество записей в столбце name*/
                FROM public.vacancies /*Из таблицы vacancies*/
                WHERE 
                    (
                    LOWER(name) like '%data scientist%' OR /*Где значения в столбце, приведённые к нижнему регистру, содержат комбинацию символов 'data scientist'*/
                    LOWER(name) like '%data science%' OR /*или 'data science'*/
                    LOWER(name) like '%исследователь данных%' OR /*или 'исследователь данных'*/
                    (name not like '%HTML%' and name like '%ML%') OR /*или 'ML', но не 'HTML'*/
                    LOWER(name) like '%machine learning%' OR /*или 'machine learning'*/
                    LOWER(name) like '%машинн%обучен%' /*или 'машинн...обучен'*/
                    )
                    AND /*а также*/
                    (
                    LOWER(name)  like '%junior%' OR /*Где значения в столбце, приведённые к нижнему регистру, содержат комбинацию символов 'junior'*/
                    experience = 'Нет опыта' OR /*или значения в столбце experience 'Нет опыта'*/                
                    employment = 'Стажировка' /*или значения в столбце employment 'Стажировка'*/  
                    )
            '''

In [40]:
# результат запроса
jun_DS = pd.read_sql_query(query_6_2, connection)
jun_DS

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# текст запроса
query_6_3 = f'''SELECT /*Запрос, аналогичный запросу 6_2*/
                    COUNT(name)
                FROM public.vacancies 
                WHERE 
                    (
                    LOWER(name) like '%data scientist%' OR 
                    LOWER(name) like '%data science%' OR
                    LOWER(name) like '%исследователь данных%' OR
                    (name not like '%HTML%' and name like '%ML%') OR
                    LOWER(name) like '%machine learning%' OR
                    LOWER(name) like '%машинн%обучен%'
                    )
                    AND
                    (
                    key_skills LIKE '%SQL%' OR /*Где значения в столбце key_skills содержат комбинацию символов 'SQL'*/
                    key_skills LIKE '%postgres%' /*или 'postgres'*/
                    )
            '''

In [42]:
# результат запроса
DS_SQL = pd.read_sql_query(query_6_3, connection)
DS_SQL

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# текст запроса
query_6_4 = f'''SELECT /*Запрос, аналогичный запросу 6_3*/
                    COUNT(name)
                FROM public.vacancies 
                WHERE 
                    (
                    LOWER(name) like '%data scientist%' OR 
                    LOWER(name) like '%data science%' OR
                    LOWER(name) like '%исследователь данных%' OR
                    (name not like '%HTML%' and name like '%ML%') OR
                    LOWER(name) like '%machine learning%' OR
                    LOWER(name) like '%машинн%обучен%'
                    )
                    AND key_skills LIKE '%Python%' /*Где значения в столбце key_skills содержат комбинацию символов 'Python'*/
            '''

In [44]:
# результат запроса
DS_python = pd.read_sql_query(query_6_4, connection)
DS_python

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
# текст запроса
query_6_5 = f'''WITH Temp AS( /*Формируем временный запрос под названием Temp*/
                    SELECT 
                        key_skills, /*Выбираем столбец key_skills*/
                        LENGTH(key_skills) - LENGTH(REPLACE(key_skills, '\t', '')) + 1 AS skills_number /*Считаем разницу между длиной записей в столбце key_skills*/ 
                                                        /*и длиной записей в том же столбце без табуляций, к разнице прибавляем 1; присваиваем название столбцу skills_number*/
                    FROM public.vacancies /*Из таблицы vacancies*/
                    WHERE /*Далее повторяем условие проверки принадлежности к профессии дата-сайентиста из запроса 6_2*/
                    (
                    LOWER(name) like '%data scientist%' OR 
                    LOWER(name) like '%data science%' OR
                    LOWER(name) like '%исследователь данных%' OR
                    (name not like '%HTML%' and name like '%ML%') OR
                    LOWER(name) like '%machine learning%' OR
                    LOWER(name) like '%машинн%обучен%'
                    )
                    )
                    
                SELECT
                    ROUND(AVG(skills_number), 2) /*Считаем среднее по столбцу skills_number*/
                FROM Temp /*Из таблицы Temp*/
    '''

In [46]:
# результат запроса
DS_avg_skills = pd.read_sql_query(query_6_5, connection)
DS_avg_skills

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
# текст запроса
query_6_6 = f'''SELECT 
                    experience, /*Выбираем столбец experience*/
                    ROUND(AVG(COALESCE((salary_from + salary_to) / 2, salary_from, salary_to))) avg_salary /*Считаем среднее из суммы записей в столбцах salary_from и salary_to,*/
                                    /*делённой на 2, предварительно очистив записи от нулевых значений при помощи функции COALESCE; результат округляем до двух знаком после запятой*/
                FROM public.vacancies /*Из таблицы vacancies*/
                WHERE /*Далее повторяем условие проверки принадлежности к профессии дата-сайентиста из запроса 6_2*/
                    (
                    LOWER(name) like '%data scientist%' OR 
                    LOWER(name) like '%data science%' OR
                    LOWER(name) like '%исследователь данных%' OR
                    (name not like '%HTML%' and name like '%ML%') OR
                    LOWER(name) like '%machine learning%' OR
                    LOWER(name) like '%машинн%обучен%'
                    )
                GROUP BY experience /*Группируем по столбцу experience*/
    '''

In [48]:
# результат запроса
DS_avg_salary = pd.read_sql_query(query_6_6, connection)
DS_avg_salary

,experience,avg_salary
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


***

Предметный анализ показал следующее.

1. 1771 вакансия из 49197 (3,6%) так или иначе относятся к работе с данными.

2. 51 вакансия (0,1% от общего количества и 2,9% от связанных с работой с данными) подходят для начинающего дата-сайентиста. Числа из п.1 и 2 могут показаться незначительными. Но это только в относительном выражении. В абсолютном же исчислении они довольно внушительны и однозначно указывают на востребованность профессии дата-сайентиста на рынке труда.

3. В 201 вакансии, подходящей для дата-сайентиста, требуются знания SQL или PostgreSQL, что неудивительно, поскольку получение данных из базы - один из ключевых навыков для представителей данной профессии.

4. То же самое можно сказать и о навыках владения языком программирования Python, который требуется в 351 вакансии. Полученные данные необходимо обработать для построения моделей машинного обучения. Python - наиболее популярный инструмент для этой цели.

5. Вообще дата-сайентист должен обладать целым спектром навыков, на что недвусмысленно указывает и другой показатель, полученный в ходе исследования - 6,4. Именно столько в среднем навыков указывается в вакансиях для дата-сайентистов.

6. Безусловно, такая профессия должна и оплачиваться соответствующе. В ходе исследования было установлено, что средняя зарплата, указываемая в вакансиях для дата-сайентистов, составляет 74.643 руб. для начинающих специалистов, 139.675 руб. - для специалистов с опытом от 1 года до 3 лет и 243.115 руб. - для специалистов с опытом работы от 3 до 6 лет.

# Общий вывод по проекту

Проведём несколько дополнительных исследований.

Выясним, какие вакансии преобладают на рынке труда.

In [49]:
query_7_1 = f'''SELECT
                    DISTINCT name, /*Выбираем уникальные записи в столбце name*/
                    COUNT(name) /*Считаем количество записей в столбце name*/
                FROM public.vacancies /*Из таблицы vacancies*/
                GROUP BY name /*Группируем по столбцу name*/
                ORDER BY 2 DESC /*Сортируем по убыванию по столбцу 2*/
            '''

In [50]:
range_vacancies = pd.read_sql_query(query_7_1, connection)
range_vacancies.head(10) # Выведем топ-10 вакансий.

,name,count
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


Как видно из запроса, среди топ-10 вакансий на рынке труда более всего требуются системные администраторы. Помимо этого, очень востребованы специальности, связанные с программированием и аналитикой. Данный факт легко объясним. От работы системного администратора зависит бесперебойная работа как отдела, так и компании в целом. В современном мире IT-технологий стабильная работа компьютерной техники имеет решающее значение. Что касается программирования, то это основа IT-индустрии. Программисты будут востребованы всегда, разница может только заключаться в востребованности тех или иных языков программирования. Аналитика - важная составляющая бизнеса. От хорошей работы аналитичесвих отделов напрямую зависит доходность компании, поэтому востребованность аналитиков так высока.

Теперь определим, какие навыки наиболее востребованы на рынке труда.

In [51]:
# Вначале создадим запрос, который выведет нам все комбинации навыков, упоминаемых в таблице vacancies
query_7_2 = f'''SELECT 
                    key_skills /*Выбираем столбец key_skills*/
                FROM public.vacancies /*Из таблицы vacancies*/
            '''

In [52]:
# Выведем результат запроса
all_skills = pd.read_sql_query(query_7_2, connection)
all_skills.dropna(inplace=True) # Сразу удалим пустые строки
all_skills

,key_skills
0,Пользователь ПК\tРабота в команде\tРемонт ноут...
1,Средства криптографической защиты информации\t...
2,Spring Framework\tSQL\tHibernate ORM\tJava\tGit
3,Spring Framework\tSQL\tHibernate ORM\tJava\tGit
5,Корректура текстов\tГрамотность\tРабота в кома...
...,...
49191,Управление проектами\tДеловая коммуникация\tОр...
49192,Электронный документооборот\t1С: Предприятие 8...
49193,Оптимизация бизнес-процессов\tСтратегическое м...
49194,Организаторские навыки\tУправление персоналом\...


In [53]:
# Разделим записи в каждой строке по знаку табуляции при помощи лямбда-функции:
all_skills['key_skills'] = all_skills['key_skills'].apply(lambda x: x.split('\t'))

# Создадим пустой словарь. Ключами в нём будут навыки, а значениями - количество упоминаний этих навыков в таблице vacancies:
skills = {}


# Напишем функцию, принимающую на вход ячейку столбца и ищущую в нём отдельные навыки:
def find_skill(col):
    for elem in col: # Проверяем каждый элемент списка из ячейки
        if elem in skills.keys(): # Если элемент присутствует в качестве ключа в словаре skills, то значение под этим ключом увеличиваем на 1
            skills[elem] += 1
        else: # Если элемент отсутствует в качестве ключа в словаре skills, то заносим в словарь такой ключ со значением 1
            skills[elem] = 1
    return skills # Фукция возвращает словарь


all_skills['key_skills'].apply(find_skill) # Применяем функцию к столбцу key_skills
# Для удобства преобразуем словарь skills в таблицу skills_table
skills_table = pd.DataFrame(list(skills.items()), columns = ['Навык', 'Кол-во'])
skills_table = skills_table.sort_values(by='Кол-во', ascending=False) # Отсортируем по убыванию по столбцу "Кол-во"
skills_table.head(10) # Выведем топ-10 навыков

,Навык,Кол-во
15,SQL,6977
18,Git,4731
67,Английский язык,4525
1,Работа в команде,3923
30,Linux,3747
22,JavaScript,3625
78,Python,3445
57,Управление проектами,2764
43,1С программирование,2647
48,PostgreSQL,2452


Как видим, знание SQL является самым востребованным. Оно почти в полтора раза превышает второй по востребованности навык - Git, а если объединить навыки SQL и PostgreSQL, то разница между топ-1 и топ-2 навыми будет практически двукратной. Безусловно, данный вывод весьма приблизителен, поскольку, как мы видим, в запросе объединились как хард-, так и софт-скиллы (например, работа в команде), но примерные представления о востребованности навыков на рынке труда данный запрос, вне всякого сомнения, даёт.

Теперь поробуем определить ключевые навыки для дата-сайентиста.

In [54]:
# Вначале создадим запрос, аналогичный предыдущему, но с фильтром для дата-сайентиста, который мы применялм ранее:
query_7_3 = f'''SELECT 
                    key_skills 
                FROM public.vacancies 
                WHERE 
                    (
                    LOWER(name) like '%data scientist%' OR 
                    LOWER(name) like '%data science%' OR 
                    LOWER(name) like '%исследователь данных%' OR 
                    (name not like '%HTML%' and name like '%ML%') OR 
                    LOWER(name) like '%machine learning%' OR 
                    LOWER(name) like '%машинн%обучен%'
                    )
            '''

In [55]:
# Выведем результат запроса
DS_skills = pd.read_sql_query(query_7_3, connection)
DS_skills.dropna(inplace=True) # Сразу удалим пустые строки
DS_skills

,key_skills
0,Python\tSQL\tРабота в команде\tOpenCV\tML
1,Oracle Pl/SQL\tБазы данных\tMachine Learning\t...
2,Python\tOpenCV\tTensorflow\tComputer Vision\tН...
3,Python\tFlask\tLinux\tAWS\tGit
4,Python\tSQL\tNLP\tScikit-learn\tMachine Learning
...,...
475,Python\tNumpy\tPyTorch\tPandas\tМатематический...
476,Python\tScikit-learn\tPandas\tGit\tNumpy
477,Qt\tООП\tC++\tMS Visual Studio\tРазработка ком...
478,SCALA\tSQL\tМатематический анализ\tСтатистичес...


In [56]:
# Разделим записи в каждой строке по знаку табуляции при помощи лямбда-функции:
DS_skills['key_skills'] = DS_skills['key_skills'].apply(lambda x: x.split('\t'))

# Создадим пустой словарь. Ключами в нём будут навыки, а значениями - количество упоминаний этих навыков в таблице vacancies:
skills_for_DS = {}


# Напишем функцию, принимающую на вход ячейку столбца и ищущую в нём отдельные навыки:
def find_DS_skill(col):
    for elem in col: # Проверяем каждый элемент списка из ячейки
        if elem in skills_for_DS.keys(): # Если элемент присутствует в качестве ключа в словаре skills_for_DS, то значение под этим ключом увеличиваем на 1
            skills_for_DS[elem] += 1
        else: # Если элемент отсутствует в качестве ключа в словаре skills_for_DS, то заносим в словарь такой ключ со значением 1
            skills_for_DS[elem] = 1
    return skills_for_DS # Фукция возвращает словарь


DS_skills['key_skills'].apply(find_DS_skill) # Применяем функцию к столбцу key_skills
# Для удобства преобразуем словарь skills_for_DS в таблицу DS_skills_table
DS_skills_table = pd.DataFrame(list(skills_for_DS.items()), columns = ['Навык', 'Кол-во'])
DS_skills_table = DS_skills_table.sort_values(by='Кол-во', ascending=False) # Отсортируем по убыванию по столбцу "Кол-во"
DS_skills_table.head(10) # Выведем топ-10 навыков

,Навык,Кол-во
0,Python,348
1,SQL,191
7,Machine Learning,114
9,Git,66
25,Математическая статистика,62
40,Data Analysis,54
45,Data Science,52
63,Pandas,52
4,ML,49
43,Английский язык,48


Как видно из таблицы, для дата-сайентистов ключевым навыком становится знание Python, а SQL отходит на второе место с чуть мнее, чем двукратным отрывом. Помимо этого, часто упоминаются такие навыки, как Machine Learning, Git, Data Analysis/Science. Кроме того, важное место приобретает знание математической статистики. Происходит это по той причине, что для дата-сайентиста принципиальное значение имеет умение обрабатывать полученные данные и выстраивать по ним машинное обучение.

Меня, как жителя города Энгельса (город-спутник Саратова), естественно, интересует рынок труда в Саратове и Энгельсе. Я позволю мебе воспользоваться случаем и провести соответствующие исследования.

Вначале выясним, какие вакансии преобладают в Саратове и Энгельсе.

In [57]:
query_7_4 = f'''SELECT
                    DISTINCT v.name, /*Выбираем уникальные записи в столбце name*/
                    COUNT(v.name) /*Считаем количество записей в столбце name*/
                FROM public.vacancies v /*Из таблицы vacancies с алиасом v*/
                JOIN public.areas a ON v.area_id = a.id /*Присоединяем таблицу areas с алиасом а по столбцам v.area_id и a.id*/
                WHERE a.name = 'Саратов' OR a.name = 'Энгельс' /*где значение в солбце a.name является "Саратов" или "Энгельс"*/
                GROUP BY v.name /*Группируем по столбцу name*/
                ORDER BY 2 DESC /*Сортируем по убыванию по столбцу 2*/
            '''

In [58]:
# Выведем результат запроса
saratov_vacancies = pd.read_sql_query(query_7_4, connection)
saratov_vacancies.head(10) # Выведем топ-10 вакансий.

,name,count
0,Программист 1С,29
1,Системный администратор,25
2,Специалист технической поддержки,14
3,Системный аналитик,10
4,Программист 1C,8
5,Инженер-программист,5
6,Ведущий инженер-программист,4
7,Инженер технической поддержки,4
8,Программист,4
9,Технический директор,4


Итак, самыми популярными вакансиями в Энгельсе и Саратове фвляются Программист 1С, Системный администратор, Специалист технической поддержки и Системный аналитик. Остальные вакансии представлены в незначительном количестве.

Проанализируем, на какую зарплату можно претендовать в Саратове и Энгельсе.

In [59]:
query_7_5 = f'''SELECT
                    ROUND(AVG(COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to))) avg_salary /*Считаем среднюю зарплату по образцу запроса 6_6*/
                FROM public.vacancies v /*Из таблицы vacancies с алиасом v*/
                JOIN public.areas a ON v.area_id = a.id /*Присоединяем таблицу areas с алиасом а по столбцам v.area_id и a.id*/
                WHERE a.name = 'Саратов' OR a.name = 'Энгельс' /*где значение в солбце a.name является "Саратов" или "Энгельс"*/
            '''

In [60]:
# Выведем результат запроса
saratov_salary = pd.read_sql_query(query_7_5, connection)
saratov_salary

,avg_salary
0,69638.0


Итак, средняя зарплата, на которую может претендовать соискатель из Саратова или Энгельса, составляет почти 70 тыс. руб.

Насколько это меньше или больше в сравнении с другими регионами? Создадим запрос, в котором сравним среднюю зарплату в моём родном регионе с зарплатой в Москве, Санкт-Петербурге, российиских городах-миллионниках и прочих регионах.

In [63]:
query_7_6 = f'''SELECT
    'Саратов и Энгельс' region,
    ROUND(AVG(COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to))) avg_salary /*Считаем среднюю зарплату по образцу предыдущего запроса*/
FROM public.vacancies v /*Из таблицы vacancies*/
JOIN public.areas a ON v.area_id = a.id /*Присоединяем таблицу areas с алиасом а по столбцам v.area_id и a.id*/
WHERE a.name = 'Саратов' OR a.name = 'Энгельс' /*где значение в солбце a.name является "Саратов" или "Энгельс"*/

UNION ALL /*Оператор объединения*/

SELECT /*Далее делаем аналогичные запросы, только меняя регионы*/
    'Москва' region,
    ROUND(AVG(COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to))) avg_salary 
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id 
WHERE a.name = 'Москва' /*где значение в солбце a.name является "Москва"*/

UNION ALL

SELECT
    'Санкт-Петербург' region,
    ROUND(AVG(COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to))) avg_salary 
FROM public.vacancies v 
JOIN public.areas a ON v.area_id = a.id 
WHERE a.name = 'Санкт-Петербург' /*где значение в солбце a.name является "Санкт-Петербург"*/

UNION ALL

SELECT
    'Города-миллионники' region,
    ROUND(AVG(COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to))) avg_salary 
FROM public.vacancies v 
JOIN public.areas a ON v.area_id = a.id 
WHERE a.name in {cities_millionairs} /*где значение в солбце a.name входит в кортеж cities_millionairs*/

UNION ALL

SELECT
    'Прочие регионы' region,
    ROUND(AVG(COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to))) avg_salary 
FROM public.vacancies v 
JOIN public.areas a ON v.area_id = a.id 
WHERE /*где значение в солбце a.name не входит ни в один из предыдущих запросов*/
    a.name != 'Санкт-Петербург' AND 
    a.name != 'Москва' AND
    a.name != 'Саратов' AND
    a.name != 'Энгельс' AND
    a.name not in {cities_millionairs} 
    
ORDER BY 2 DESC /*Сортируем по убыванию по второму столбцу*/
            '''

In [64]:
# Выведем результат запроса
saratov_salary_in_compair = pd.read_sql_query(query_7_6, connection)
saratov_salary_in_compair

,region,avg_salary
0,Москва,134140.0
1,Санкт-Петербург,104410.0
2,Города-миллионники,97980.0
3,Прочие регионы,73754.0
4,Саратов и Энгельс,69638.0


Как видно из запроса, уровень заработной платы в Саратове и Энгельсе ниже, чем в прочих регионах и почти в два раза уступает соответствующим показателям в Москве.

# <center> Вывод

В базе данных есть в общей сложности 1771 вакансия, так или иначе связанная с обработкой данных и 51, подходящая для претендентов на позицию Data Scientist. Как показал анализ, для будущего дата-сайентиста ключевыми навыками, как и ожидалось, являются владение Python, SQL и машинным обучением. Больше всего вакансий представлено в городах Москва и Санкт_Петербург. После них, если брать только росийские города, идут города-миллионники. И в этих же городах самые высокие зарплаты. Такая тенденция свойственна для России не только в сфере IT-индустрии, но и в прочих сферах деятельности.

Будущему соискателю необходимо знать, что наибольшие шансы у него получить работу будут в том случае, если он готов работать полный рабочий день по полной занятости, а также, если опыт работы у него от 1 года до 3 лет. Очевидно, что такая тенденция сложилась в силу того, что компании не хотят нанимать кандидатов без опыта работы, а опыт работы хотя бы от 1 года может свидетельствовать о том, что соискатель обладает необходимыми навыками на базовом уровне, понимает, что от него требуется и знает, как работать в команде.

Среди работодателей, имеющихся в базе данных, преобладают крупные IT-компании, которые имеют филиалы в различных городах России. Объясняется это тем, что работа в крупной IT-компании имеет объективные преимущества. Это и более высокие зарплатные ожидания, и различные сферы деятельности, в которых можно проявить себя, и более высокие перспективы карьерного роста. Работа в крупных IT-компаниях может быть просто более интересной, хотя, без сомнения, это дело вкуса. Для дата-сайентиста работа в крупной компании является более преспективной, поскольку именно в такой компании могут проводиться крупные и масштабные проекты в области исследования данных и машинного обучения.

Что касается Саратовского региона, то в плане работы в сфере IT он представляется не очень перспективным. Здесь не такой большой выбор вакансий и не самые высокие зарплатные перспективы. 

В целом проведённые исследования только подтвердили общую тенденцию, характерную для современного этапа развития экономики не только в России, но и во всём мире: концентрация высокотехнологических отраслей вокруг крупных центров. Именно здесь открываются более высокие перспективы как для карьерного роста, так и для роста благосостояния работников. На мой взгляд, именно такая тенденция будет преобладать в ближайшем будущем, а требования к опыту работы будут падать в связи с оттоком специалистов в области IT-технологий и, как следствие, необходимостью заменить их менее опытными кандидатами.

In [65]:
# Закрываем соединение:
connection.close()